In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../input/health-insurance-cross-sell-prediction/train.csv')

In [ ]:
t_df = pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv')

In [ ]:
t_df.head()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().mean()

In [ ]:
df.describe()

In [ ]:
df['Vehicle_Age'].value_counts()

In [ ]:
df['Response'].value_counts()

In [ ]:
t_df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
df['Gender'] = le.fit_transform(df['Gender'])
t_df['Gender'] = le.transform(t_df['Gender'])

In [ ]:
df['Vehicle_Age'] = le.fit_transform(df['Vehicle_Age'])
t_df['Vehicle_Age'] = le.transform(t_df['Vehicle_Age'])

In [ ]:
df['Vehicle_Damage'] = le.fit_transform(df['Vehicle_Damage'])
t_df['Vehicle_Damage'] = le.transform(t_df['Vehicle_Damage'])

In [ ]:
df.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.countplot(x='Response',data=df)
plt.show()

In [ ]:
df['Response'].value_counts(normalize=True)*100

In [ ]:
ext = df[df['Response'] == 1]

In [ ]:
train_final = pd.concat([df,ext,ext,ext,ext,ext,ext])

In [ ]:
train_final['Response'].value_counts(normalize=True)*100

In [ ]:
import seaborn as sns
sns.countplot(x='Response',data=train_final)
plt.show()

In [ ]:
dff = train_final
dff.corr().round(2)

In [ ]:
sns.heatmap(dff.corr())

In [ ]:
   
cat_features = dff[[ 'Vehicle_Damage', 'Previously_Insured', 'Gender','Vehicle_Damage', 'Vehicle_Age', 'Driving_License']].columns
for i in cat_features:
    sns.barplot(x="Response",y=i,data=dff)
    plt.title(i+" by "+"Response")
    plt.show()

In [ ]:
X = dff.drop(['Response','id'],axis = 1)
y = dff['Response']

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier().fit(X,y)

In [ ]:
dt.feature_importances_

In [ ]:
X.columns

In [ ]:
pd.DataFrame(dt.feature_importances_,index=['Gender', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured','Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium','Policy_Sales_Channel', 'Vintage']).plot.bar()

In [ ]:
X_new = X.drop(['Gender','Driving_License','Vehicle_Age'],axis = 1)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_new,y,test_size = 0.33,random_state = 42)

In [ ]:
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from xgboost import XGBClassifier

In [ ]:
dt = DecisionTreeClassifier(random_state=42).fit(x_train,y_train)

In [ ]:
dt.score(x_test,y_test)

In [ ]:
dt_pred = dt.predict(x_test)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
print('DT_Score :',roc_auc_score(y_test,dt_pred))

In [ ]:
forest = RandomForestClassifier(random_state=42).fit(x_train,y_train)
forest.score(x_test,y_test)

In [ ]:
forest_pred = forest.predict(x_test)
print('Forest_Score :',roc_auc_score(y_test,forest_pred))

In [ ]:
grad = GradientBoostingClassifier().fit(x_train,y_train)
grad_pred = grad.predict(x_test)
grad.score(x_test,y_test)

In [ ]:
print('GradientBoosting_Score :',roc_auc_score(y_test,grad_pred))

In [ ]:
ada = AdaBoostClassifier().fit(x_train,y_train)
ada_pred = ada.predict(x_test)

In [ ]:
ada.score(x_test,y_test)
print('AdaBoost_Score :',roc_auc_score(y_test,ada_pred))

In [ ]:
xgb = XGBClassifier().fit(x_train,y_train)
xgb_pred = xgb.predict(x_test)

In [ ]:
xgb.score(x_test,y_test)

In [ ]:
print('xgb_Score :',roc_auc_score(y_test,xgb_pred))

# Decision Tree is givng best results

In [ ]:
dt = DecisionTreeClassifier(random_state=42).fit(x_train,y_train)

In [ ]:
dt.score(x_test,y_test)

In [ ]:
print('ROC_AUC_Score :',roc_auc_score(y_test,dt_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, dt_pred).ravel()
confusion_matrix

In [ ]:
#Predicting proba
y_pred_prob = dt.predict(x_test)
from sklearn.metrics import roc_curve
# Generate ROC curve values: fpr, tpr, thresholds
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Plot ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

In [ ]:
from sklearn import metrics
cols = ['Model', 'ROC Score', 'Precision Score', 'Recall Score','Accuracy Score','Kappa Score']
models_report = pd.DataFrame(columns = cols)

tmp5 = pd.Series({'Model': " Decision Tree",
                 'ROC Score' : metrics.roc_auc_score(y_test, dt_pred),
                 'Precision Score': metrics.precision_score(y_test, dt_pred),
                 'Recall Score': metrics.recall_score(y_test, dt_pred),
                 'Accuracy Score': metrics.accuracy_score(y_test, dt_pred),
                 'Kappa Score':metrics.cohen_kappa_score(y_test, dt_pred)})

model5_report = models_report.append(tmp5, ignore_index = True)
model5_report

In [ ]:
t_df = t_df[X_new.columns]

In [ ]:
scc = StandardScaler().fit(X_new)

In [ ]:
X_new_sc = scc.transform(X_new)
t_sc = scc.transform(t_df)

In [ ]:
dtt = DecisionTreeClassifier(random_state=42).fit(X_new_sc,y)

In [ ]:
pred = dtt.predict(t_sc)

In [ ]:
pd.Series(pred).to_csv('Prediction.csv')